In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from account import Binance
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import cvxpy as cp
from utils.logging import get_logger
from utils.data_helper import *
from utils.db import *
from strategy_v3.Strategy import ExchangeArbitrageStrategy

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = "{:,.4f}".format
warnings.filterwarnings('ignore')

# Script to find all Binance traded CCY in USD

There are many currency which doesn't have an explicit USD price, we need to normalize all CCY to USD price for trading strategy 

e.g.

- Determine the trade size in USD

- Determine the best bid/ask size in USD

- Determine the VWAP price in order book up to my USD trade size 
    - If I trade $1,000 USD from best ask price, what would be my average price?

- Measure the volume in USD

In [45]:
# init all symbols fundemental data (only need to load once)        
client = Binance().get_client()
exch_info = client.get_exchange_info()
price = client.get_orderbook_tickers()

In [95]:
df_symbols = pd.DataFrame(exch_info['symbols'])
df_symbols = df_symbols[df_symbols['status'] == 'TRADING']
df_symbols = df_symbols[['symbol', 'status', 'baseAsset', 'quoteAsset']]
df_price = pd.DataFrame(price)
df_symbols = pd.merge(df_symbols, df_price, how='inner', on='symbol', validate='1:1')
df_symbols = Binance.format_output(df_symbols)
df_symbols['price'] = (df_symbols['bidPrice'] + df_symbols['askPrice']) / 2

In [124]:
from utils.ccy import ExchangeRateGraph
G = ExchangeRateGraph()

for _, row in df_symbols.iterrows():
    baseAsset = row['baseAsset']
    quoteAsset = row['quoteAsset']
    px = row['price']
    G.add_edge(baseAsset, quoteAsset, px)
    G.add_edge(quoteAsset, baseAsset, 1/px)

In [129]:
x = G.get_all_rates("USDT")